In [4]:
import requests
import json

# Twitch credentials 
CLIENT_ID = '1ppjb55ykv0m8z83e7kqu47qlyasp3'
ACCESS_TOKEN = 'w02ysg11je3m0uclfyfl6gmjttxs4l'

# Define the base URL for IGDB API
BASE_URL = 'https://api.igdb.com/v4/games'

# Function to retrieve games based on genre and save each batch of 500 records into a separate JSON file
def fetch_games_by_genre(genre_ids, fields, limit=500, max_games=3000):
    headers = {
        'Client-ID': CLIENT_ID,
        'Authorization': f'Bearer {ACCESS_TOKEN}',
        'Accept': 'application/json'
    }
    
    offset = 0
    batch_num = 1
    total_games = 0

    while True:
        query = f'''
        fields {fields};
        where genres = ({", ".join(map(str, genre_ids))});
        limit {min(limit, max_games - total_games)};
        offset {offset};
        '''
        
        response = requests.post(BASE_URL, headers=headers, data=query)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        games = response.json()
        if not games:
            break
        
        for game in games:
            if 'genres' in game:
                game['genre_names'] = get_genre_names(game['genres'])

        filename = f'games_batch_{batch_num}.json'
        with open(filename, 'w') as json_file:
            json.dump(games, json_file, indent=4)
        print(f"Saved {len(games)} games to {filename}")

        total_games += len(games)
        if total_games >= max_games:
            print(f"Reached maximum number of games ({max_games})")
            break

        offset += len(games)
        batch_num += 1

        if len(games) < limit:
            break

    print(f"Total games fetched: {total_games}")
# New function to get genre names for given genre IDs
def get_genre_names(genre_ids):
    genres_url = 'https://api.igdb.com/v4/genres'
    headers = {
        'Client-ID': CLIENT_ID,
        'Authorization': f'Bearer {ACCESS_TOKEN}',
        'Accept': 'application/json'
    }
    
    genre_query = f'''
    fields name;
    where id = ({", ".join(map(str, genre_ids))});
    '''
    
    response = requests.post(genres_url, headers=headers, data=genre_query)
    
    if response.status_code != 200:
        print(f"Error fetching genre names: {response.status_code} - {response.text}")
        return []

    try:
        genres = response.json()
        return [genre['name'] for genre in genres]
    except json.JSONDecodeError:
        print("Error: Unable to parse JSON response for genre names")
        return []
    except KeyError as e:
        print(f"Error: Missing key in genre response - {e}")
        return []

# Fetch genre IDs for Adventure and Casual
def get_genre_ids():
    genres_url = 'https://api.igdb.com/v4/genres'
    headers = {
        'Client-ID': CLIENT_ID,
        'Authorization': f'Bearer {ACCESS_TOKEN}',
        'Accept': 'application/json'
    }
    
    genre_query = '''
    fields id, name;
    where name = "Adventure" | name = "Casual";
    '''
    
    response = requests.post(genres_url, headers=headers, data=genre_query)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        return []

    try:
        genres = response.json()
        genre_ids = [genre['id'] for genre in genres if genre['name'] in ['Adventure', 'Casual']]
        return genre_ids
    except json.JSONDecodeError:
        print("Error: Unable to parse JSON response")
        return []
    except KeyError as e:
        print(f"Error: Missing key in response - {e}")
        return []

def main():
    # Define the fields to retrieve
    fields = 'name, genres, summary, storyline, rating, url'

    # Get the genre IDs for "Adventure" and "Casual"
    genre_ids = get_genre_ids()

    if genre_ids:
        print(f"Genre IDs found: {genre_ids}")

        # Fetch games with those genres and save batches to JSON files
        fetch_games_by_genre(genre_ids, fields, limit=500, max_games=2500)
    else:
        print("No genre IDs found for Adventure or Casual.")

if __name__ == '__main__':
    main()

Genre IDs found: [31]
Saved 500 games to games_batch_1.json
Saved 500 games to games_batch_2.json
Saved 500 games to games_batch_3.json
Saved 500 games to games_batch_4.json
Saved 500 games to games_batch_5.json
Reached maximum number of games (2500)
Total games fetched: 2500
